# Connect to google drive to obtain the image data

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!ln -s '/content/drive/My Drive/Low Field MRI' /mydrive

Mounted at /content/drive
ln: failed to create symbolic link '/mydrive/Low Field MRI': Operation not supported


In [19]:
def unzip_file(zf_path, destination):
    print(f"Unzipping file: {zf_path}.zip...\n")
    !unzip -o -q '{zf_path}' -d '{destination}'
    !rm {zf_path}.zip
    print("Done!")

In [21]:
import os
import shutil

img_folder = 'axial_10days_images'

!rm -r "{img_folder}"
!cp -r "/mydrive/Datasets/SG Data/{img_folder}.zip" "/content"

unzip_file(img_folder,'')

Unzipping file: axial_10days_images.zip...

Done!


# Import necessary libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from pprint import pprint

In [ ]:
def save_img(img, dst_folder, day, session, i):
    if not os.path.exists(dst_folder):
        os.mkdir(dst_folder)

    if not os.path.exists(os.path.join(dst_folder, day)):
        os.mkdir(f"{dst_folder}/{day}")

    if not os.path.exists(os.path.join(dst_folder, day, session)):
        os.mkdir(f"{dst_folder}/{day}/{session}")

    filename = os.path.join(dst_folder, day, session, str(i))
    np.save(filename, img)

In [ ]:
datasets_folder = '/mydrive/Datasets/SG_Data/Axial_10days'
datasets = os.listdir(datasets_folder)
datasets = [dataset for dataset in datasets if dataset[:3]=='Day']

dst_folder = "/content/axial_10days_images"

for day in datasets:
    print(day)
    day_path = os.path.join(datasets_folder, day)
    sessions = os.listdir(day_path)
    
    for session in sessions:
        print(f"\t{session}")
        session_path = os.path.join(day_path, session)
        
        # Read folder
        dataFolder = f'{session_path}/3DTSE'
        raw_folder = '1'
        noiseScan = 0
        freqDrift = -60  # Frequency drift

        '''Read k-space and reconstruct image'''

        reconImage_raw, kSpace_raw, scanParams_raw, trajectory_raw = cPP.prepData(dataFolder, raw_folder, noiseScan, freqDrift)

        fig = plt.figure(figsize=(20,6))
        for i in range(25):
            img = np.absolute(reconImage_raw[:,:,i])
            save_img(img, dst_folder, day, session, i)
            ax = fig.add_subplot(4,8,i+1)
            ax.imshow(img, cmap='gray')
            ax.axis("off")

        fig.subplots_adjust(hspace=0.35)
        # plt.gcf().set_dpi(300)
        # plt.savefig('images/noise')
        # plt.gcf().set_dpi(80)
        plt.show()
        
        # # Display
        # OrthoSlicer3D(np.abs(reconImage_raw)).show()
        # break
    # break

FileNotFoundError: ignored

In [ ]:
!cp -r "/content/axial_10days_images" "/mydrive/Datasets/SG Data"